# Demonstration of Pointing-Object Usage 
 - 20190315
 - MJP
 - Starting to develop "Pointing" Objects 
 - Will have some/lots of inheritance from previous projects 

In [1]:
# Standard imports 
import numpy as np 
# MPC imports 
import MPC_library as MPCL
import phys_const as PHYS
import mpchecker 
import classes as Classes
import angles as ANG
import params 

/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Some of the results returned below will depend on the assumed HP scale for the preprocessing
 - E.g. the HP # for the center of the pointing ...
 - At the time of evaluation the assumed parameters are ...

In [2]:
print("Healpix: Nside", params.hp_nside)
print("Healpix: Npix", params.hp_npix)
print("Healpix: Area [deg^2]", params.hp_area)
print("Healpix: Side [deg]", params.hp_side)

Healpix: Nside 128
Healpix: Npix 196608
Healpix: Area [deg^2] 0.20982341130279172
Healpix: Side [deg] 0.45806485490898746


# Define and populate a "Pointing" Object
 - A unique pointing can be defined by: (a) obscode, (b) date/time, and (c) (RA,Dec) of the center
 - Note that printing the pointing returns the obscode_JDutc_
 - Note that these can be initialized with or without specifying the size (diameter) of the FoV

In [3]:
obsCode = 'G96'
JDutc   = 2458360.12345
RA_center, Dec_center = (123.45, 67.89) # <<-- Quantities assumed to be in degrees 

# Initialize with NO field-of-view specification
P = Classes.Pointing(obsCode, JDutc, RA_center, Dec_center) 
print(type(P), P)
print() 

# Initialize with FoV information 
FoV_degrees = 1.0 # <<-- Assumed to be a diameter of FoV in degrees
P = Classes.Pointing(obsCode, JDutc, RA_center, Dec_center, FoV_degrees)
print(type(P), P)
print(P.FOV_rad)

<class 'classes.Pointing'> G96_2458360.12345_123.45_67.89

<class 'classes.Pointing'> G96_2458360.12345_123.45_67.89
0.017453292519943295


# Examine the defined pointing object ... 

In [4]:
# These quantities are defined by the act of loading 
print("P.obsCode = %r "%P.obsCode )
print("P.JDutc = %r "%P.JDutc )
print("P.RADEC = " , P.RADEC )
print() 
print("P.FOV = " , P.FOV )
print("P.FOV_rad = " , P.FOV_rad )
print() 

# Internal calculation of unit vector 
print("Before: P.UV = " , P.UV )
P.calc_UV()
print("After: P.UV = " , P.UV , type(P.UV))
print()

# Check that the conversion back from RADEC to unit-vector gives the expected numbers 
recalcRADEC = ANG.unit2radec(P.UV)
print("Reconversion: RADEC: P.RADEC = " , recalcRADEC )
print("All close: ", np.allclose(P.RADEC , recalcRADEC))

P.obsCode = 'G96' 
P.JDutc = 2458360.12345 
P.RADEC =  (123.45, 67.89)

P.FOV =  1.0
P.FOV_rad =  0.017453292519943295

Before: P.UV =  None
After: P.UV =  [-0.20746736  0.31404409  0.92646295] <class 'numpy.ndarray'>

Reconversion: RADEC: P.RADEC =  (array([123.45]), array([67.89]))
All close:  False


# It is useful to Calculate the Healpix for the FoV of a pointing
 - The HP of the *CENTER* can be calculated irrespective of whether FOV information has been specified
 - But if we initialize a Pointing with NO FoV information, we need to add in the FoV info if we want to be able to calculate a reliable list of HP covered by the pointing
 - In the absence of any information on the FoV, if "coverage" is requested I am deliberately quitting so that the user is forced to contemplate their mistakes in life
 - N.B. in the examples below I did *NOT* have to explicitly calculate the unitvector, as I am nice enough to make that happen internally if/when required (see "HP_of_center" inside "Pointing" class) 

In [5]:
# Look at the HP occupied by the pointing-*CENTER*
# - This works irrespective of whether FoV is defined 

# Initialize with *NO* FoV information 
P = Classes.Pointing(obsCode, JDutc, RA_center, Dec_center) 
print(P.HP_of_center(), P.HPcenter) 
print() 

# Initialize with FoV information 
FoV_degrees = 1.0 # <<-- Assumed to be a diameter of FoV in degrees
P = Classes.Pointing(obsCode, JDutc, RA_center, Dec_center, FoV_degrees)
print(P.HP_of_center(), P.HPcenter) 

31174 31174

31174 31174


In [6]:
# Look at the HP occupied by the entire FoV of the pointing
# Initialize with FoV information 
FoV_degrees = 1.0 # <<-- Assumed to be a diameter of FoV in degrees
P = Classes.Pointing(obsCode, JDutc, RA_center, Dec_center, FoV_degrees)
print(P.HP_covered_by_FoV(), P.HPfov, type(P.HPfov) ) 

[31169 31170 31171 31172 31173 31174 31175 31176 31177 31180 31181] [31169 31170 31171 31172 31173 31174 31175 31176 31177 31180 31181] <class 'numpy.ndarray'>


# It is useful for the pointing to be able to calculate / store the heliocentric position of the observatory 
 - Calculating the ECLIPTIC coords is done via a calculation of the EQUATORIAL coords

In [7]:
# Demonstrate that at this point neither "helio_eq_posn" nor "helio_ec_posn" exist in "P"
print( hasattr(P , 'helio_eq_posn') , hasattr(P , 'helio_ec_posn') )
      
# Call the func to calc_heliocentric_position_of_observatory_in_ecliptic_coords
P.calc_heliocentric_position_of_observatory_in_ecliptic_coords()
      
# Demonstrate that now we have both "helio_eq_posn" and "helio_ec_posn" exist in "P"
print("P.helio_eq_posn = ",P.helio_eq_posn)
print("P.helio_ec_posn = ",P.helio_ec_posn)

False False
P.helio_eq_posn =  [ 0.92233908 -0.37724584 -0.1635308 ]
P.helio_ec_posn =  [ 9.22339082e-01 -4.11165108e-01  2.31351528e-05]


# Select the Subset of an RA_, DEC_ list which are actually in the Field-of-View of a Pointing


In [8]:
# Here are some sample RAs & DECs to check against the pointing
RA_, DEC_  = [123.45, 123.46, 12.45, 123.54,],[67.89, 67.99, 6.89, 68.00]
    
# Get the angle between the RA_,DEC_ and the center of the pointing
indiceesWithinFoV = P.select_RADEC_within_FoV(RA_, DEC_)

# Examine the output
print(len(indiceesWithinFoV[0]), len(indiceesWithinFoV[0])<=len(RA_))
print(indiceesWithinFoV)


3 True
(array([0, 1, 3]),)
